# 01 Data preparation

Processing xlsx files from data folder into suitable inputs and generate other input files

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
cell_line ='BC3C'
data_dir = f"/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/blca_publication_OUTPUT_LINCS/00_outputs_2020_{cell_line}/"
info_dir = data_dir
out_dir = f"/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/blca_publication_OUTPUT_bmra/blca_publication_OUTPUT_bmra_{cell_line}/00_outputs_2020_{cell_line}"

#os.makedirs(info_dir, exist_ok = True)
os.makedirs(out_dir, exist_ok = True)

## Modules

Load data about modules and drugs.

In [4]:
### DATA
### remove whitespaces in names (modules), remove duplicates

modules_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "modules", index_col = 0)
# display(modules_df)

selected_modules = modules_df.index.tolist()
print(len(selected_modules), ' - Size after reading')

# remove duplicates
selected_modules = modules_df.index.unique().tolist()
print(len(selected_modules), ' - Size after remove duplicates')

# remove whitespaces in modules' names 
selected_modules = [d.strip() for d in selected_modules]

print('Selected modules list: ', len(selected_modules), selected_modules)


10  - Size after reading
10  - Size after remove duplicates
Selected modules list:  10 ['CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'PI3K', 'FGFR', 'TOP2A', 'p53', 'Estrogen', 'Androgen']


In [5]:
### DATA
### remove whitespaces in names (modules, drugs), remove duplicates
### check the dimensions of the indicator IC50 1 uM = 1000 nM
### copy-paste as values, numbers, no formulas

IC50_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "IC50s")
IC50_df.drop(columns=['Unnamed: 3','Unnamed: 4'],inplace=True)

print(len(IC50_df.index), ' - Size after reading')
# display(IC50_df)

# rename
IC50_df = IC50_df.rename(columns = {"IC50, uM": "IC50"})

# manually correcting value IC50, Example  for IOX2 -> 30 nM  
#IC50_df.loc[IC50_df.index == 'IOX2', IC50_df.columns == 'IC50'] = 30/1000

# remove non-selected modules, modules' names with whitespaces or empty
IC50_df = IC50_df[IC50_df.Module.isin(selected_modules)]

print(len(IC50_df.index), ' - Size after remove modules')
# display(IC50_df)

# remove duplicates 
# Considering certain columns is optional. 
# Indexes, including time indexes are ignored.
IC50_df = IC50_df.drop_duplicates()

print(len(IC50_df.index), ' - Size after remove duplicates')
display(IC50_df)

39  - Size after reading
39  - Size after remove modules
39  - Size after remove duplicates


,Drug,Module,IC50
0,flufenamic-acid,Androgen,3.00000
1,nandrolone,Androgen,9.00000
2,oxandrolone,Androgen,190.30000
3,testosterone-enanthate,Androgen,200.00000
4,testosterone-propionate,Androgen,124.00000
5,JNJ-7706621,CDK1,0.02700
6,PHA-793887,CDK1,0.18000
7,roscovitine,CDK2,2.00000
8,alvocidib,CDK4_6,0.12000
9,palbociclib,CDK4_6,0.04500


In [6]:
modules = IC50_df.Module.unique().tolist()

print('IC50_df  modules list: ', len(modules), modules)
print()
print('Selected modules list: ', len(selected_modules), selected_modules)

### CHECK
print()
print('CHECK: ', len(selected_modules),'=?', len(modules))

n_modules = len(modules)


IC50_df  modules list:  10 ['Androgen', 'CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'Estrogen', 'FGFR', 'PI3K', 'p53', 'TOP2A']

Selected modules list:  10 ['CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'PI3K', 'FGFR', 'TOP2A', 'p53', 'Estrogen', 'Androgen']

CHECK:  10 =? 10


In [7]:
drugs = IC50_df.Drug.tolist()
print(len(drugs), ' - Size after reading')

# remove duplicates
drugs = IC50_df.Drug.unique().tolist()
print(len(drugs), ' - Size after remove duplicates')

# remove whitespaces in drugs' names (necessary for some)
drugs = [d.strip() for d in drugs]

# remove duplicates after remove whitespaces
drugs = list(set(drugs))
print(len(drugs), ' - Size after remove duplicates without whitespaces')

print('Drugs list: ', len(drugs), drugs)

n_drugs = len(drugs)

39  - Size after reading
39  - Size after remove duplicates
39  - Size after remove duplicates without whitespaces
Drugs list:  39 ['testosterone-enanthate', 'ponatinib', 'estradiol-cypionate', 'oxandrolone', 'PI-103', 'afatinib', 'roscovitine', 'flufenamic-acid', 'testosterone-propionate', 'SAR405838', 'gefitinib', 'RITA', 'sorafenib', 'AZD-8055', 'HLI-373', 'palbociclib', 'masitinib', 'erlotinib', 'JNJ-7706621', 'NVP-BEZ235', 'serdemetan', 'taselisib', 'vandetanib', 'PHA-793887', 'idarubicin', 'GDC-0349', 'AMG-232', 'AS-605240', 'dienestrol', 'daunorubicin', 'raloxifene', 'mitoxantrone', 'lapatinib', 'KU-0063794', 'nutlin-3', 'alvocidib', 'epirubicin', 'nandrolone', 'LY-294002']


## L1000 meta data

Get sig_id for selected drugs.

In [8]:
sig_info_df = pd.read_excel(os.path.join(data_dir, f"sig_info_2020_{cell_line}.xlsx"), index_col = 0)

display(sig_info_df)

,cell,plate,time,level_3_samples,samples_number,pert_type,pert_drug,targets,targets_number,dose,dose_float
level_5_sig_id,,,,,,,,,,,
ASG002_BC3C_24H:A03,BC3C,ASG002,24 h,ASG002_BC3C_24H_X1_B35:A03,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_BC3C_24H:A04,BC3C,ASG002,24 h,ASG002_BC3C_24H_X1_B35:A04,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_BC3C_24H:A05,BC3C,ASG002,24 h,ASG002_BC3C_24H_X1_B35:A05,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_BC3C_24H:A06,BC3C,ASG002,24 h,ASG002_BC3C_24H_X1_B35:A06,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
ASG002_BC3C_24H:J13,BC3C,ASG002,24 h,ASG002_BC3C_24H_X1_B35:J13,1,ctl_vehicle,DMSO,DMSO_No_target,0,0 uM,0.00
...,...,...,...,...,...,...,...,...,...,...,...
MOAR012_BC3C_24H:P20,BC3C,MOAR012,24 h,MOAR012_BC3C_24H_X1_B36:P20,1,trt_cp,BAY-61-3606,NaN,0,3.33 uM,3.33
MOAR012_BC3C_24H:P21,BC3C,MOAR012,24 h,MOAR012_BC3C_24H_X1_B36:P21,1,trt_cp,BAY-61-3606,NaN,0,1.11 uM,1.11
MOAR012_BC3C_24H:P22,BC3C,MOAR012,24 h,MOAR012_BC3C_24H_X1_B36:P22,1,trt_cp,ethaverine,NaN,0,10 uM,10.00


In [54]:
# now filtering so only the required drugs are present
sig_info_df = sig_info_df.loc[sig_info_df.pert_drug.isin(drugs)]

# here's what we have now
display(sig_info_df)

,cell,plate,time,level_3_samples,samples_number,pert_type,pert_drug,targets,targets_number,dose,dose_float
level_5_sig_id,,,,,,,,,,,
ASG002_CAL29_24H:A10,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A10,1,trt_cp,taselisib,PIK3CA,1,10 uM,10.00
ASG002_CAL29_24H:A11,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A11,1,trt_cp,taselisib,PIK3CA,1,1.11 uM,1.11
ASG002_CAL29_24H:A12,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A12,1,trt_cp,taselisib,PIK3CA,1,0.12 uM,0.12
ASG002_CAL29_24H:A19,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A19,1,trt_cp,AS-605240,PIK3CG,1,10 uM,10.00
ASG002_CAL29_24H:A20,CAL29,ASG002,24 h,ASG002_CAL29_24H_X1_B35:A20,1,trt_cp,AS-605240,PIK3CG,1,1.11 uM,1.11
...,...,...,...,...,...,...,...,...,...,...,...
MOAR011_CAL29_24H:C11,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:C11,1,trt_cp,testosterone-enanthate,AR,1,3.33 uM,3.33
MOAR011_CAL29_24H:C12,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:C12,1,trt_cp,testosterone-enanthate,AR,1,1.11 uM,1.11
MOAR011_CAL29_24H:J10,CAL29,MOAR011,24 h,MOAR011_CAL29_24H_X1_B35:J10,1,trt_cp,nandrolone,"AR, CYP19A1, MAOA, MAOB, NISCH",5,10 uM,10.00


Manually remove  few inhibition from data set, since it does differ from the other data points.

inhib_to_filter = "PF-03758309"
dose_to_filter = 10

id_to_filter = sig_info_df[
    np.logical_and(
        sig_info_df.pert_drug == inhib_to_filter,
        sig_info_df.dose_float == dose_to_filter,
    )
].index.values

print(id_to_filter)

sig_info_df = sig_info_df[~sig_info_df.index.isin(id_to_filter)]
display(sig_info_df)

inhib_to_filter = "roscovitine"
dose_to_filter = 3.33

id_to_filter = sig_info_df[
    np.logical_and(
        sig_info_df.pert_drug == inhib_to_filter,
        sig_info_df.dose_float == dose_to_filter,
    )
].index.values

print(id_to_filter)

sig_info_df = sig_info_df[~sig_info_df.index.isin(id_to_filter)]
display(sig_info_df)

In [9]:
exp_ids = sig_info_df.index.unique().tolist()
print('Experiments ids list: ', len(exp_ids))

n_experiments = len(exp_ids)

Experiments ids list:  1971


Confirm data by checking the drugs of interest against the filtered L1000 meta data.

In [10]:
print(f"Number of drugs of interest:\t{len(drugs)}")
#print(f'Number of drugs in L1000 data:\t{len(sig_info_df.value_counts("drugs"))}')

#sig_info_df.value_counts("drugs")

Number of drugs of interest:	39


## L1000 data

In [11]:
Data_norm_df = pd.read_excel(os.path.join(data_dir, f"Data_norm_2020_{cell_line}.xlsx"), index_col = 0)
display(Data_norm_df)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
ASG002_BC3C_24H:A03,-0.191254,-0.055246,0.039596,-0.256266,-0.040419,-0.590523,-0.159396,-0.074319,0.457981,0.409608,...,0.543203,0.494266,-0.011923,-0.225931,0.285054,-0.775246,0.166031,-0.024873,0.238723,0.284204
ASG002_BC3C_24H:A04,-0.265754,-0.317496,0.118696,-0.136665,-0.301569,-0.403023,0.124804,-0.036470,0.311931,0.660457,...,-0.565096,-0.088634,0.122977,-0.047931,0.141804,0.129054,-0.028819,-0.028773,-0.253627,-0.752646
ASG002_BC3C_24H:A05,-0.181954,-0.081597,-0.210304,1.559535,-0.019019,-0.457423,0.071404,0.074080,-0.356119,0.498808,...,0.226104,-0.228034,-0.121023,-0.075331,-0.133146,0.355054,0.022831,-0.084073,0.283123,-0.894896
ASG002_BC3C_24H:A06,0.033446,0.042404,-0.150154,-0.093165,0.053180,-0.053823,0.087704,0.167681,-0.601569,0.383308,...,-0.608596,-0.228835,0.072777,0.082970,-0.570996,2.847754,-0.211670,-0.067273,0.081723,0.338704
ASG002_BC3C_24H:J13,0.204446,0.180704,0.089096,-0.054666,0.053381,0.044877,-0.277396,-0.157419,0.535681,-3.933493,...,-0.318397,0.122265,-0.134323,-0.088931,-0.067996,-0.515847,-0.005069,0.067527,0.002223,0.204904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOAR012_BC3C_24H:P20,0.647151,0.211700,-0.979200,0.597350,-0.375751,0.388300,0.394524,0.120151,-0.166775,-1.129125,...,-1.598475,-0.552750,0.515151,0.120800,0.082675,0.529700,0.383225,-0.207225,2.268450,-1.248500
MOAR012_BC3C_24H:P21,0.171800,0.046300,-0.145550,-0.295150,0.030849,0.420951,0.222075,0.179800,0.274724,-0.423975,...,-1.650575,0.203600,-0.003250,-0.064800,-0.037675,0.076499,0.201825,0.416875,0.287450,-0.971700
MOAR012_BC3C_24H:P22,0.648700,0.058749,-0.031700,0.408249,-0.753950,0.332200,-0.357525,-0.107650,-0.213575,0.074225,...,0.127625,0.031600,0.103250,-0.249600,0.046375,1.486200,0.440325,0.090075,-0.031650,-0.944300
MOAR012_BC3C_24H:P23,0.090499,-0.469300,-0.611800,0.873550,-0.788450,-0.097199,-0.366575,-0.490600,-0.624675,-0.009275,...,0.054676,-0.596050,0.084600,0.444700,0.431375,-0.921501,0.044926,0.716076,-0.000900,-1.106700


In [12]:
Data_norm_df = Data_norm_df[Data_norm_df.index.isin(exp_ids)]

# arrange experiments in same order as in list
Data_norm_df["sort_col"] = Data_norm_df.index.map({val: i for i, val in enumerate(exp_ids)})
Data_norm_df = Data_norm_df.sort_values("sort_col")
Data_norm_df = Data_norm_df.drop("sort_col", axis = 1)

# transpose
Data_norm_df = Data_norm_df.T

display(Data_norm_df)

,ASG002_BC3C_24H:A03,ASG002_BC3C_24H:A04,ASG002_BC3C_24H:A05,ASG002_BC3C_24H:A06,ASG002_BC3C_24H:J13,ASG002_BC3C_24H:J14,ASG002_BC3C_24H:J15,ASG002_BC3C_24H:J16,ASG002_BC3C_24H:J17,ASG002_BC3C_24H:J18,...,MOAR012_BC3C_24H:P15,MOAR012_BC3C_24H:P16,MOAR012_BC3C_24H:P17,MOAR012_BC3C_24H:P18,MOAR012_BC3C_24H:P19,MOAR012_BC3C_24H:P20,MOAR012_BC3C_24H:P21,MOAR012_BC3C_24H:P22,MOAR012_BC3C_24H:P23,MOAR012_BC3C_24H:P24
AARS,-0.191254,-0.265754,-0.181954,0.033446,0.204446,0.408096,0.343246,0.294746,0.097946,0.174596,...,0.359300,0.235600,-0.160699,0.351300,0.900100,0.647151,0.171800,0.648700,0.090499,0.204801
ABCB6,-0.055246,-0.317496,-0.081597,0.042404,0.180704,0.170904,0.264104,0.679953,0.212804,0.060254,...,0.111150,-0.022400,-0.814200,0.043200,0.710800,0.211700,0.046300,0.058749,-0.469300,-1.388700
ABCC5,0.039596,0.118696,-0.210304,-0.150154,0.089096,0.326696,0.346296,0.872796,-0.025904,0.393696,...,-1.173200,-0.649050,2.041700,-0.674200,-0.611500,-0.979200,-0.145550,-0.031700,-0.611800,-0.283400
ABCF1,-0.256266,-0.136665,1.559535,-0.093165,-0.054666,-0.468565,-0.439466,-0.008215,-0.174766,0.021435,...,1.054650,0.868550,1.379700,0.669800,1.291850,0.597350,-0.295150,0.408249,0.873550,0.744750
ABCF3,-0.040419,-0.301569,-0.019019,0.053180,0.053381,0.304181,0.209981,-0.061119,0.364280,0.550581,...,-1.016601,-0.464351,-1.356300,-0.434350,-0.968850,-0.375751,0.030849,-0.753950,-0.788450,-1.431150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF395,-0.775246,0.129054,0.355054,2.847754,-0.515847,1.230454,-2.410447,-1.559446,-0.790896,-1.058396,...,1.914200,2.336899,2.295700,1.586300,2.131400,0.529700,0.076499,1.486200,-0.921501,0.088650
ZNF451,0.166031,-0.028819,0.022831,-0.211670,-0.005069,0.182881,0.017630,0.327481,-0.145819,-0.140719,...,0.354325,0.487625,0.551625,0.276926,0.518675,0.383225,0.201825,0.440325,0.044926,-0.242075
ZNF586,-0.024873,-0.028773,-0.084073,-0.067273,0.067527,0.023977,0.063227,0.108677,-0.018573,0.139427,...,0.188375,0.048375,0.228976,0.191275,-0.328225,-0.207225,0.416875,0.090075,0.716076,0.195575
ZNF589,0.238723,-0.253627,0.283123,0.081723,0.002223,-0.123027,-0.165777,0.007073,0.153373,-0.119577,...,-0.677950,-0.111700,-0.455550,0.018050,-0.006250,2.268450,0.287450,-0.031650,-0.000900,-0.038300


In [13]:
genes = Data_norm_df.index.tolist()
print('Landmark genes list: ', len(genes), genes)

n_genes = len(genes)

Landmark genes list:  978 ['AARS', 'ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1', 'ACAA1', 'ACAT2', 'ACBD3', 'ACD', 'ACLY', 'ACOT9', 'ADAM10', 'ADAT1', 'ADGRE5', 'ADGRG1', 'ADH5', 'ADI1', 'ADO', 'ADRB2', 'AGL', 'AKAP8', 'AKAP8L', 'AKR7A2', 'AKT1', 'ALAS1', 'ALDH7A1', 'ALDOA', 'ALDOC', 'AMDHD2', 'ANKRD10', 'ANO10', 'ANXA7', 'APBB2', 'APOE', 'APP', 'APPBP2', 'ARFIP2', 'ARHGAP1', 'ARHGEF12', 'ARHGEF2', 'ARID4B', 'ARID5B', 'ARL4C', 'ARNT2', 'ARPP19', 'ASAH1', 'ASCC3', 'ATF1', 'ATF5', 'ATF6', 'ATG3', 'ATMIN', 'ATP11B', 'ATP1B1', 'ATP2C1', 'ATP6V0B', 'ATP6V1D', 'AURKA', 'AURKB', 'AXIN1', 'B4GAT1', 'BACE2', 'BAD', 'BAG3', 'BAMBI', 'BAX', 'BCL2', 'BCL7B', 'BDH1', 'BECN1', 'BHLHE40', 'BID', 'BIRC2', 'BIRC5', 'BLCAP', 'BLMH', 'BLVRA', 'BMP4', 'BNIP3', 'BNIP3L', 'BPHL', 'BRCA1', 'BTK', 'BUB1B', 'BZW2', 'C2CD2', 'C2CD2L', 'C2CD5', 'C5', 'CAB39', 'CALM3', 'CALU', 'CAMSAP2', 'CANT1', 'CAPN1', 'CARMIL1', 'CASC3', 'CASK', 'CASP10', 'CASP2', 'CASP3', 'CASP7', 'CAST', 'CAT', 'CBLB', 'CBR1

## Inhibitor concentrations, IC50, and perturbation matrices

In [14]:
inhib_conc_matrix = np.zeros((n_modules, n_experiments))
ic50_matrix = np.ones((n_modules, n_experiments))
gamma_matrix = np.zeros((n_modules, n_experiments))

In [15]:

for i, module in enumerate(modules):
    drugs_for_module = IC50_df.Drug[IC50_df.Module == module].tolist()
    for drug in drugs_for_module:
        # get IC50 for this drug
        ic50 = IC50_df.IC50[IC50_df.Drug == drug].values
#       gamma = IC50_df.Gamma[IC50_df.Drug == drug].values
        print(drug, ic50)
        assert ic50.size == 1
#       assert gamma.size == 1
        # get experiments with this drug
        exp_with_drug = sig_info_df.index[sig_info_df.pert_drug == drug].tolist()
        print(exp_with_drug) 
        for exp_id in exp_with_drug:
            j = exp_ids.index(exp_id)
            print(j)
            # extract inhibitor concentration
            inhib_conc = sig_info_df.dose_float[sig_info_df.index == exp_id].values
            assert inhib_conc.size == 1
            # insert values in matrices
            inhib_conc_matrix[i, j] = inhib_conc.item()
            ic50_matrix[i, j] = ic50.item()
#           gamma_matrix[i, j] = gamma.item()


flufenamic-acid [3.]
['MOAR008_BC3C_24H:L01', 'MOAR008_BC3C_24H:L02', 'MOAR008_BC3C_24H:L03']
623
624
625
nandrolone [9.]
['MOAR011_BC3C_24H:J10']
1555
oxandrolone [190.3]
['MOAR011_BC3C_24H:C01', 'MOAR011_BC3C_24H:C02', 'MOAR011_BC3C_24H:C03']
1475
1476
1477
testosterone-enanthate [200.]
['MOAR011_BC3C_24H:C10', 'MOAR011_BC3C_24H:C11']
1484
1485
testosterone-propionate [124.]
['MOAR010_BC3C_24H:L19', 'MOAR010_BC3C_24H:L20', 'MOAR010_BC3C_24H:L21']
1360
1361
1362
JNJ-7706621 [0.027]
['ASG002_BC3C_24H:N13', 'ASG002_BC3C_24H:N14', 'ASG002_BC3C_24H:N15']
323
324
325
PHA-793887 [0.18]
['ASG002_BC3C_24H:L01', 'ASG002_BC3C_24H:L02', 'ASG002_BC3C_24H:L03']
264
265
266
roscovitine [2.]
['ASG002_BC3C_24H:E22', 'ASG002_BC3C_24H:E23', 'ASG002_BC3C_24H:E24']
132
133
134
alvocidib [0.12]
['ASG002_BC3C_24H:F04', 'ASG002_BC3C_24H:F05', 'ASG002_BC3C_24H:F06']
138
139
140
palbociclib [0.045]
['ASG002_BC3C_24H:P16', 'ASG002_BC3C_24H:P17', 'ASG002_BC3C_24H:P18']
371
372
373
afatinib [0.03]
['ASG002_BC3C_

In [16]:
# transform matrices into pandas dfs for export with row and column names
inhib_conc_df = pd.DataFrame(inhib_conc_matrix, index = modules, columns = exp_ids)
ic50_df = pd.DataFrame(ic50_matrix, index = modules, columns = exp_ids)
# gamma_df = pd.DataFrame(gamma_matrix, index = modules, columns = exp_ids)

# create binary perturbation matrix
pert_df = pd.DataFrame(
    np.where(inhib_conc_matrix != 0, 1, 0),
    index = inhib_conc_df.index,
    columns = inhib_conc_df.columns,
)

In [17]:
display(ic50_df)
# display(gamma_df)
display(inhib_conc_df)
display(pert_df)

,ASG002_BC3C_24H:A03,ASG002_BC3C_24H:A04,ASG002_BC3C_24H:A05,ASG002_BC3C_24H:A06,ASG002_BC3C_24H:J13,ASG002_BC3C_24H:J14,ASG002_BC3C_24H:J15,ASG002_BC3C_24H:J16,ASG002_BC3C_24H:J17,ASG002_BC3C_24H:J18,...,MOAR012_BC3C_24H:P15,MOAR012_BC3C_24H:P16,MOAR012_BC3C_24H:P17,MOAR012_BC3C_24H:P18,MOAR012_BC3C_24H:P19,MOAR012_BC3C_24H:P20,MOAR012_BC3C_24H:P21,MOAR012_BC3C_24H:P22,MOAR012_BC3C_24H:P23,MOAR012_BC3C_24H:P24
Androgen,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CDK1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CDK2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CDK4_6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EGFR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Estrogen,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
FGFR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
PI3K,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
p53,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TOP2A,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


,ASG002_BC3C_24H:A03,ASG002_BC3C_24H:A04,ASG002_BC3C_24H:A05,ASG002_BC3C_24H:A06,ASG002_BC3C_24H:J13,ASG002_BC3C_24H:J14,ASG002_BC3C_24H:J15,ASG002_BC3C_24H:J16,ASG002_BC3C_24H:J17,ASG002_BC3C_24H:J18,...,MOAR012_BC3C_24H:P15,MOAR012_BC3C_24H:P16,MOAR012_BC3C_24H:P17,MOAR012_BC3C_24H:P18,MOAR012_BC3C_24H:P19,MOAR012_BC3C_24H:P20,MOAR012_BC3C_24H:P21,MOAR012_BC3C_24H:P22,MOAR012_BC3C_24H:P23,MOAR012_BC3C_24H:P24
Androgen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CDK1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CDK2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CDK4_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EGFR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Estrogen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FGFR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PI3K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TOP2A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,ASG002_BC3C_24H:A03,ASG002_BC3C_24H:A04,ASG002_BC3C_24H:A05,ASG002_BC3C_24H:A06,ASG002_BC3C_24H:J13,ASG002_BC3C_24H:J14,ASG002_BC3C_24H:J15,ASG002_BC3C_24H:J16,ASG002_BC3C_24H:J17,ASG002_BC3C_24H:J18,...,MOAR012_BC3C_24H:P15,MOAR012_BC3C_24H:P16,MOAR012_BC3C_24H:P17,MOAR012_BC3C_24H:P18,MOAR012_BC3C_24H:P19,MOAR012_BC3C_24H:P20,MOAR012_BC3C_24H:P21,MOAR012_BC3C_24H:P22,MOAR012_BC3C_24H:P23,MOAR012_BC3C_24H:P24
Androgen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK4_6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EGFR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Estrogen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FGFR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PI3K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
p53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TOP2A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Global responses for DPD modules

According to our discussion, $R$ for DPD vectors can not be calculated with the same formula as for pathway activities. Instead we are assuming:

\begin{equation}
R_{DPD},j = DPD = STV_{DPD} \cdot Data_j
\end{equation}

In [18]:
# load STV data frame
STVs = pd.read_excel(os.path.join(info_dir, "ALL_DATA_2020_Jing.xlsx"), sheet_name = "STV", index_col = 0)
STV_df = pd.DataFrame(np.zeros((len(Data_norm_df.index), 3)), index = Data_norm_df.index, columns = STVs.columns)
STV_df.loc[STVs.index] = STVs

display(STV_df)

,blca_invasiveness,blca_oncogenesis,blca_survival
AARS,0.016052,0.078487,0.000000
ABCB6,0.001612,0.027125,-2.744978
ABCC5,-0.016742,0.002121,0.000000
ABCF1,-0.038990,-0.028959,-0.555168
ABCF3,0.023288,0.036671,0.000000
...,...,...,...
ZNF395,0.000000,0.000000,0.000000
ZNF451,0.011450,0.013403,0.000000
ZNF586,0.000000,0.000000,0.000000
ZNF589,0.000000,0.000000,0.000000


In [19]:
# create empty DPD data frame
DPD_df = pd.DataFrame(
    np.zeros((len(Data_norm_df.columns), len(STV_df.columns))),
    index = Data_norm_df.columns,
    columns = STV_df.columns,
)

# populate
for exp_id in DPD_df.index:
    for state in STV_df.columns:
        DPD_df.loc[exp_id, state] = np.dot(Data_norm_df.T.loc[exp_id], STV_df.loc[:, state])

display(DPD_df)

,blca_invasiveness,blca_oncogenesis,blca_survival
ASG002_BC3C_24H:A03,0.224169,0.302763,-2.305911
ASG002_BC3C_24H:A04,0.150151,-0.135368,-3.117183
ASG002_BC3C_24H:A05,0.167952,-0.014359,-2.260927
ASG002_BC3C_24H:A06,0.328579,-0.094210,-6.709615
ASG002_BC3C_24H:J13,-0.582389,-0.243571,0.612093
...,...,...,...
MOAR012_BC3C_24H:P20,1.639600,-0.777955,3.452730
MOAR012_BC3C_24H:P21,0.843686,-0.200156,0.625054
MOAR012_BC3C_24H:P22,0.530906,0.472079,-1.294036
MOAR012_BC3C_24H:P23,0.684520,-0.052636,-7.776345


In [20]:
# transform to R global
R_global_DPD_df = DPD_df.T
display(R_global_DPD_df)

,ASG002_BC3C_24H:A03,ASG002_BC3C_24H:A04,ASG002_BC3C_24H:A05,ASG002_BC3C_24H:A06,ASG002_BC3C_24H:J13,ASG002_BC3C_24H:J14,ASG002_BC3C_24H:J15,ASG002_BC3C_24H:J16,ASG002_BC3C_24H:J17,ASG002_BC3C_24H:J18,...,MOAR012_BC3C_24H:P15,MOAR012_BC3C_24H:P16,MOAR012_BC3C_24H:P17,MOAR012_BC3C_24H:P18,MOAR012_BC3C_24H:P19,MOAR012_BC3C_24H:P20,MOAR012_BC3C_24H:P21,MOAR012_BC3C_24H:P22,MOAR012_BC3C_24H:P23,MOAR012_BC3C_24H:P24
blca_invasiveness,0.224169,0.150151,0.167952,0.328579,-0.582389,0.090671,-0.280056,-0.054662,-0.301350,-0.032421,...,0.794299,0.649506,-0.550905,1.315166,0.853185,1.639600,0.843686,0.530906,0.684520,-0.184652
blca_oncogenesis,0.302763,-0.135368,-0.014359,-0.094210,-0.243571,0.322869,-0.202286,0.984030,0.188662,0.383745,...,0.437785,-0.296724,-0.957768,-0.311339,0.329503,-0.777955,-0.200156,0.472079,-0.052636,0.325686
blca_survival,-2.305911,-3.117183,-2.260927,-6.709615,0.612093,3.529396,3.110927,3.094277,3.581075,3.071644,...,7.527461,0.666810,3.964668,-5.086449,-11.084706,3.452730,0.625054,-1.294036,-7.776345,-22.252404


## Save outputs

In [21]:
# save metadata as pickle
all_metadata = {
    "modules": modules,
    "n_modules": n_modules,
    "drugs": drugs,
    "n_drugs": n_drugs,
    "exp_ids": exp_ids,
    "n_experiments": n_experiments,
    "genes": genes,
    "n_genes": n_genes,
}

print(all_metadata)

with open(os.path.join(out_dir, "metadata.pickle"), "wb") as f:
    pickle.dump(all_metadata, f, protocol = pickle.HIGHEST_PROTOCOL)

{'modules': ['Androgen', 'CDK1', 'CDK2', 'CDK4_6', 'EGFR', 'Estrogen', 'FGFR', 'PI3K', 'p53', 'TOP2A'], 'n_modules': 10, 'drugs': ['testosterone-enanthate', 'ponatinib', 'estradiol-cypionate', 'oxandrolone', 'PI-103', 'afatinib', 'roscovitine', 'flufenamic-acid', 'testosterone-propionate', 'SAR405838', 'gefitinib', 'RITA', 'sorafenib', 'AZD-8055', 'HLI-373', 'palbociclib', 'masitinib', 'erlotinib', 'JNJ-7706621', 'NVP-BEZ235', 'serdemetan', 'taselisib', 'vandetanib', 'PHA-793887', 'idarubicin', 'GDC-0349', 'AMG-232', 'AS-605240', 'dienestrol', 'daunorubicin', 'raloxifene', 'mitoxantrone', 'lapatinib', 'KU-0063794', 'nutlin-3', 'alvocidib', 'epirubicin', 'nandrolone', 'LY-294002'], 'n_drugs': 39, 'exp_ids': ['ASG002_BC3C_24H:A03', 'ASG002_BC3C_24H:A04', 'ASG002_BC3C_24H:A05', 'ASG002_BC3C_24H:A06', 'ASG002_BC3C_24H:J13', 'ASG002_BC3C_24H:J14', 'ASG002_BC3C_24H:J15', 'ASG002_BC3C_24H:J16', 'ASG002_BC3C_24H:J17', 'ASG002_BC3C_24H:J18', 'ASG002_BC3C_24H:O04', 'ASG002_BC3C_24H:O05', 'ASG002

In [22]:
# save doses and perturbation matrix
inhib_conc_df.to_csv(os.path.join(out_dir, "inhib_conc_annotated.csv"))
ic50_df.to_csv(os.path.join(out_dir, "ic50_annotated.csv"))
# gamma_df.to_csv(os.path.join(out_dir, "gamma_annotated.csv"))
pert_df.to_csv(os.path.join(out_dir, "pert_annotated.csv"))

In [23]:
# save log fold change L1000 data
Data_norm_df.to_csv(os.path.join(out_dir, "L1000_Data_norm_data.csv"))

In [24]:
# save R_global for DPDs
R_global_DPD_df.to_csv(os.path.join(out_dir, "R_global_DPDonly_annotated.csv"))